In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri May  5 17:53:23 2017

@author: zhouyi
"""

import glob
import os
import sys

import implicit
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.sparse


import evaluation

DATA_ROOT = '/Users/zhouyi/Documents/GraduatePJ/ml-1m'

In [2]:
unique_uid = list()
with open(os.path.join(DATA_ROOT, 'unique_uid.txt'), 'r') as f:
    for line in f:
        unique_uid.append(line.strip())
        
unique_sid = list()
with open(os.path.join(DATA_ROOT, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_songs = len(unique_sid)
n_users = len(unique_uid)

In [3]:
def load_data(csv_file, shape=(n_songs, n_users)):
    tp = pd.read_csv(csv_file)
    cols, rows = np.array(tp['uid'], dtype=np.int32), np.array(tp['sid'], dtype=np.int32)
    count = tp['rating']
    return scipy.sparse.csr_matrix((count,(rows, cols)), dtype=np.int16, shape=shape), rows, cols


train_data, rows, cols = load_data(os.path.join(DATA_ROOT, 'train.num.csv'))
# binarize the data
train_data.data = np.ones_like(train_data.data)

print 'train__data.shape:',train_data.shape
print 'train_data entries:',train_data.data.shape

vad_data, rows_vad, cols_vad = load_data(os.path.join(DATA_ROOT, 'vad.num.csv'))
# binarize the data
vad_data.data = np.ones_like(vad_data.data)
print 'vad__data.shape:',vad_data.shape
print 'vad_data entries:',vad_data.data.shape

test_data, rows_test, cols_test = load_data(os.path.join(DATA_ROOT, 'test.num.csv'))
# binarize the data
test_data.data = np.ones_like(test_data.data)
print 'test_data.shape:',test_data.shape
print 'test_data entries:',test_data.data.shape

print 'sparse according to vad_data:',100*float(vad_data.data.shape[0])/(vad_data.shape[0]*vad_data.shape[1]),"%"

train__data.shape: (2019, 4200)
train_data entries: (636096,)
vad__data.shape: (2019, 4200)
vad_data entries: (70677,)
test_data.shape: (2019, 4200)
test_data entries: (176693,)
sparse according to vad_data: 0.833474846105 %


In [4]:
#fit the WMF model
IMMOVIEmodel = implicit.als.AlternatingLeastSquares(factors=18)
IMMOVIEmodel.fit(train_data.astype(np.double))

In [5]:
'''Compute validation metric '''
def validate(model, X, vad_data):
        
    vad_recall_at_k = evaluation.Recall_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
        
    vad_map_at_k = evaluation.MAP_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
        
    vad_ndcg_at_k = evaluation.NDCG_at_k(X.T, vad_data.T,
                                                model.user_factors,
                                                model.item_factors,
                                                k=20)
    if True:
        print('\tValidation NDCG@k: %.5f' % vad_ndcg_at_k)
        print('\tValidation Recall@k: %.5f' % vad_recall_at_k)
        print('\tValidation MAP@k: %.5f' % vad_map_at_k)
        sys.stdout.flush()
            
        
    return vad_ndcg_at_k,vad_recall_at_k,vad_map_at_k

In [6]:
VAD_RECORD=validate(IMMOVIEmodel,train_data,vad_data)

	Validation NDCG@k: 0.05367
	Validation Recall@k: 0.10566
	Validation MAP@k: 0.05385


In [7]:
'''More results are summarized here

Test NDCG@k: 0.13116
Test Recall@k: 0.14797
Test MAP@k: 0.13138
'''

'More results are summarized here\n\nTest NDCG@k: 0.13116\nTest Recall@k: 0.14797\nTest MAP@k: 0.13138\n'